In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import set_option
import time
from numpy import set_printoptions
from numpy.random import random, uniform, choice
from numpy import set_printoptions, int0
from numpy import (zeros, min, log10, tanh, arctanh,
                    shape, clip, exp, pi, cos,sqrt, ravel, log)
from copy import deepcopy
import typing as t
from sklearn.linear_model import LinearRegression
from typing import Any

# <div style="color:blue">MULTIPLE REGRESSION</div>

In [2]:
# File Paths 
compression = "/home/adisa/Desktop/Working/Akinwale/compression.csv"
flexural = "/home/adisa/Desktop/Working/Akinwale/flexural.csv"
split = "/home/adisa/Desktop/Working/Akinwale/split.csv"


In [3]:
def linear_regression(path: str) -> Any:
    p = path.split("/")[-1].split(".")[0]
    df_com = pd.read_csv(path)
    df_des= df_com.describe()
    lb =  df_des.iloc[3, :-1].values
    ub = df_des.iloc[-1,:-1].values
    X = df_com.iloc[:, 1:-1].values
    model_x = df_com.iloc[::9, 1: -1].values
    Y = df_com.iloc[:, -1].values
    model = LinearRegression().fit(X, Y)
    r_sq = model.score(X, Y)
    inter = model.intercept_
    coeff = model.coef_
    print(p)
    statement = f"b0:{round(inter, 2)}, b1:{round(coeff[0], 2)}, b2:{round(coeff[1], 2)}, b3:{round(coeff[2], 2)}, b4:{round(coeff[3], 2)}, b5:{round(coeff[4], 2)}"
    return X, lb, ub

In [4]:
###################
# Compression, flexural and split
lr_com = linear_regression(compression)
lr_flex = linear_regression(flexural)
lr_split = linear_regression(split)

compression
flexural
split


In [5]:
xc, lb_c, ub_c = lr_com
xf, lb_f, ub_f = lr_flex
xs, lb_s, ub_s = lr_split

# <div style="color:green">Memory</div>

In [6]:
def memory(pol_size, d):
    lb = [0.113, 0, 0.457, 1.371, 0.113]
    ub = [1.029, 0.206, 2.286, 6.857, 0.566]
    Matrix_Harmony_Memory = np.zeros((pol_size,d))
    #print(Matrix_Harmony_Memory)
    for i in range(pol_size):
        for j in range(d):
            Matrix_Harmony_Memory[i,j] =lb[j] + np.random.random()*(ub[j]-lb[j])
    return Matrix_Harmony_Memory

In [7]:
X_memory = memory(1000, 5)
lb = [0.113, 0, 0.457, 1.371, 0.113]
ub = [1.029, 0.206, 2.286, 6.857, 0.566]

# <div style="color:green">META_HEURISTIC ALGORITHMS</div>

## <div style="color:yellow">CONSTANTS</div>


In [8]:
# harmony      
HMCR = 0.95                   #Harmony memory consideration rate
PAR = 0.3                     #Pitch Adjustment rate
bw = 1                        #bandwidth
# Fire-fly
ALPHA = 1 # randomness 
GAMMA = 0.01 #absorption coefficient
BETA0 = 1 
THETA= 0.97
LOLP = 0.1 # adjustable
# slime mould
EPSILON = 10E-10
ID_FIT = 1                   #id of fittness 
ID_WEI = 2                   #id of weight
ID_POS = 0                   # id of best position normal
ID_BEST_F= 0                 # best position fittness
ID_WORST_F = -1              # worst position
Z = 0.03
# general
num_iter = 1000 #number of iterations
runs = 5

# <div style="color:grey">OBJECTIVE FUNCTIONS</div>

In [9]:
from math import ceil
def obj_func_compression(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    comp = 34.81+409.54*ce+648.19*sba+0*fi-34.15*co+1.07*wa
    return comp

def obj_func_flexural(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    flexural = 19.49-0.14*ce-8.84*sba+0.6*fi-1.19*co+0.15*wa
    return flexural

def obj_func_split(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    split = 54.88-4.28*ce-320.83*sba+0*fi-6*co+0*wa
    return split
def generic_func(x):
     x = np.array(x).tolist()
     ce, sba, fi, co, wa = x
     res = abs(34.81+409.54*ce+648.19*sba+0*fi-34.15*co+1.07*wa)+ \
            abs(19.49-0.14*ce-8.84*sba+0.6*fi-1.19*co+0.15*wa)+ \
            abs(54.88-4.28*ce-320.83*sba+0*fi-6*co+0*wa)
     return res

# <div style="color:grey">HARMONY SEARCH</div>


In [10]:
from array import array
def harmony_search(lb: list, ub: list, harmony_mem: np.array, obj_func: t.Callable):
    # import random
    pol_size = len(harmony_mem)
    start = time.time()
    global_best_score = {}
    for run in range(runs):     
        Matrix_Harmony_Memory = harmony_mem # experimental data
        Matrix_Harmony_Fitness = []
        for k in range(pol_size):
            Matrix_Harmony_Fitness.append(obj_func(Matrix_Harmony_Memory[k,:]))
            
        set_printoptions(precision=3)
        components = []
        times = []
        best_score = []
        for it_num in range (num_iter):
            scale = np.array(ub) - np.array(lb)
            for i in range(0, pol_size): # len of the experimental da
                old_obj =Matrix_Harmony_Fitness[i]
                Random_Number = np.random.random() # Random number between zero and one
                if Random_Number > HMCR :
                    xnew = (np.array(lb) + Random_Number * scale)
                    #rescaling the features of the harmony memory
                    xnew = np.clip(xnew, lb, ub)
                    xnew = np.ravel(xnew)
                elif Random_Number <= HMCR:
                    row_indices = np.random.choice(pol_size, 1, replace= False)
                    xnew = Matrix_Harmony_Memory[row_indices,:]
                    if Random_Number <= PAR :
                        # random_state = (np.random.randn() -0.5 )
                        opt = bw * (Random_Number - 0.5)*scale
                        xnew = (xnew + opt)
                        #rescaling the features of the harmony memort
                        xnew = np.clip(xnew, lb, ub)     
                    elif Random_Number >  HMCR:
                        #rescaling the features of the harmony memory
                        xnew = np.clip(xnew, lb, ub) 
                    xnew = np.ravel(xnew)
                new_obj = obj_func(xnew)
                if new_obj < old_obj:
                    Matrix_Harmony_Memory[i,:] = xnew
                else:
                    Matrix_Harmony_Memory[i,:] = Matrix_Harmony_Memory[i,:]
            # best_values.append[Matrix_Harmony_Memory]
            min_obj = np.min([obj_func(Matrix_Harmony_Memory[i,:]) for i in range(pol_size)])
            best_score.append(min_obj)
            # components.append(Matrix_Harmony_Memory[best_score.index(min(best_score)),:])
            times.append(time.time() -start)
            print(f'Iteration {it_num} && Best Score {min_obj}')
        print("========================================================")
        global_best_score[f'best_score_run_{run}'] = best_score
        global_best_score[f'components_run_{run}'] = components
        global_best_score[f'times_run_{run}'] = times
    return Matrix_Harmony_Memory, global_best_score


In [11]:
### General
gen_res = harmony_search(lb,ub, X_memory, generic_func)

Iteration 0 && Best Score 16.967170440500983
Iteration 1 && Best Score 16.967170440500983
Iteration 2 && Best Score 16.967170440500983
Iteration 3 && Best Score 16.967170440500983
Iteration 4 && Best Score 16.967170440500983
Iteration 5 && Best Score 16.967170440500983
Iteration 6 && Best Score 16.967170440500983
Iteration 7 && Best Score 16.967170440500983
Iteration 8 && Best Score 16.967170440500983
Iteration 9 && Best Score 16.967170440500983
Iteration 10 && Best Score 16.967170440500983
Iteration 11 && Best Score 16.967170440500983
Iteration 12 && Best Score 16.967170440500983
Iteration 13 && Best Score 16.967170440500983
Iteration 14 && Best Score 16.967170440500983
Iteration 15 && Best Score 16.967170440500983
Iteration 16 && Best Score 16.967170440500983
Iteration 17 && Best Score 16.967170440500983
Iteration 18 && Best Score 16.967170440500983
Iteration 19 && Best Score 16.967170440500983
Iteration 20 && Best Score 16.967170440500983
Iteration 21 && Best Score 16.96717044050098

In [ ]:
gen_res

(array([[0.113, 0.   , 0.511, 3.765, 0.113],
        [0.113, 0.   , 0.511, 3.765, 0.113],
        [0.113, 0.   , 0.511, 3.765, 0.113],
        ...,
        [0.113, 0.   , 0.511, 3.765, 0.113],
        [0.113, 0.   , 0.511, 3.765, 0.113],
        [0.113, 0.   , 0.511, 3.765, 0.113]]),
 {'best_score_run_0': [0.14890761731134816,
   0.21889963467936724,
   0.20612410064565267,
   0.20612410064565267,
   0.20612410064565267,
   0.20612410064565267,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,
   0.2379793139496087,

In [ ]:
compression_search = harmony_search(lb_c, ub_c,xc ,obj_func_compression)
compression_search

Iteration 0 && Best Score 34.43330277588871
Iteration 1 && Best Score 34.43330277588871
Iteration 2 && Best Score 34.43330277588871
Iteration 3 && Best Score 34.43330277588871
Iteration 4 && Best Score 34.43330277588871
Iteration 5 && Best Score 34.43330277588871
Iteration 6 && Best Score 34.43330277588871
Iteration 7 && Best Score 34.43330277588871
Iteration 8 && Best Score 34.43330277588871
Iteration 9 && Best Score 34.43330277588871
Iteration 10 && Best Score 34.43330277588871
Iteration 11 && Best Score 34.43330277588871
Iteration 12 && Best Score 34.43330277588871
Iteration 13 && Best Score 34.43330277588871
Iteration 14 && Best Score 34.43330277588871
Iteration 15 && Best Score 34.43330277588871
Iteration 16 && Best Score 34.43330277588871
Iteration 17 && Best Score 34.43330277588871
Iteration 18 && Best Score 34.43330277588871
Iteration 19 && Best Score 34.43330277588871
Iteration 20 && Best Score 34.43330277588871
Iteration 21 && Best Score 34.43330277588871
Iteration 22 && Best

(array([[0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0.113, 0.   , 0.457, 1.371, 0.113],
        [0

In [ ]:
flexural_search = harmony_search(lb_f, ub_f,xf ,obj_func_flexural)
flexural_search

Iteration 0 && Best Score 10.875701903907487
Iteration 1 && Best Score 10.86217832323376
Iteration 2 && Best Score 10.833208193653856
Iteration 3 && Best Score 10.833208193653856
Iteration 4 && Best Score 10.83095091850294
Iteration 5 && Best Score 10.833208193653856
Iteration 6 && Best Score 10.82764061630369
Iteration 7 && Best Score 10.82764061630369
Iteration 8 && Best Score 10.82764061630369
Iteration 9 && Best Score 10.82764061630369
Iteration 10 && Best Score 10.82764061630369
Iteration 11 && Best Score 10.82764061630369
Iteration 12 && Best Score 10.82764061630369
Iteration 13 && Best Score 10.82764061630369
Iteration 14 && Best Score 10.82764061630369
Iteration 15 && Best Score 10.82764061630369
Iteration 16 && Best Score 10.827213432043866
Iteration 17 && Best Score 10.82764061630369
Iteration 18 && Best Score 10.82764061630369
Iteration 19 && Best Score 10.82764061630369
Iteration 20 && Best Score 10.82764061630369
Iteration 21 && Best Score 10.82764061630369
Iteration 22 &&

(array([[1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1.029, 0.206, 2.286, 6.857, 0.566],
        [1

In [ ]:
split_search = harmony_search(lb_s, ub_s, xs ,obj_func_split)
split_search

Iteration 0 && Best Score 20.696205213333222
Iteration 1 && Best Score 20.109336797413782
Iteration 2 && Best Score 20.241785659484123
Iteration 3 && Best Score 20.241785659484123
Iteration 4 && Best Score 20.241785659484123
Iteration 5 && Best Score 20.241785659484123
Iteration 6 && Best Score 20.128729241438677
Iteration 7 && Best Score 20.241785659484123
Iteration 8 && Best Score 20.241785659484123
Iteration 9 && Best Score 20.241785659484123
Iteration 10 && Best Score 20.241785659484123
Iteration 11 && Best Score 20.241785659484123
Iteration 12 && Best Score 20.133545950313128
Iteration 13 && Best Score 20.089292502734914
Iteration 14 && Best Score 20.241785659484123
Iteration 15 && Best Score 20.241785659484123
Iteration 16 && Best Score 20.241785659484123
Iteration 17 && Best Score 20.67862983084889
Iteration 18 && Best Score 20.41998629319893
Iteration 19 && Best Score 20.26689975330357
Iteration 20 && Best Score 20.26689975330357
Iteration 21 && Best Score 20.120667473430153
It

(array([[0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0.321, 0.064, 0.713, 2.139, 0.177],
        [0